In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from itertools import product
from random import shuffle
from tqdm import tqdm

from KNN_Embeddings import *

(89996, 512)


/home/pmao/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy: 0.5928969359331476
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1338
           1       0.53      1.00      0.69       847
           2       0.30      0.77      0.44       339
           3       0.51      0.93      0.66       634
           4       0.66      0.20      0.31      1035
           5       0.43      0.22      0.29       592
           6       0.86      0.24      0.37       741
           7       0.16      0.14      0.15       421
           8       0.69      0.61      0.65      1233

    accuracy                           0.59      7180
   macro avg       0.57      0.56      0.50      7180
weighted avg       0.65      0.59      0.56      7180

Confusion Matrix:
[[1234   73    2    0   22    2    2    2    1]
 [   2  843    0    0    1    0    0    1    0]
 [   0   35  260   17    0   23    0    4    0]
 [   0    4   26  587    0    1    2    1   13]
 [  44  343   53   99  210   89  

In [2]:
# Feature normalization
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
import random

class TripletDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        # Ensure labels are in a flat, 1D array
        if labels.dim() > 1:
            labels = labels.view(-1)
        self.labels = labels
        self.labels_set = set(labels.numpy())
        self.label_to_indices = {label: np.where(labels.numpy() == label)[0]
                                 for label in self.labels_set}

    def __getitem__(self, index):
        anchor = self.embeddings[index]
        anchor_label = self.labels[index].item()
        positive_index = index
        while positive_index == index:
            positive_index = random.choice(self.label_to_indices[anchor_label])
        negative_label = random.choice(list(self.labels_set - {anchor_label}))
        negative_index = random.choice(self.label_to_indices[negative_label])
        positive = self.embeddings[positive_index]
        negative = self.embeddings[negative_index]
        return anchor, positive, negative, anchor_label, negative_label

    def __len__(self):
        return len(self.embeddings)


# Create a scaler object
scaler = StandardScaler()

# Fit on training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TripletDataset(X_train_tensor, y_train_tensor)
test_dataset = TripletDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)



## Shared Sub Network

In [3]:
class SharedSubNet(nn.Module):
    def __init__(self):
        super(SharedSubNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(64, 50, kernel_size=1, stride=1)  # Using 1x1 convolutions
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        return x

## Divide and Encode module

In [4]:
class DivideAndEncode(nn.Module):
    def __init__(self, num_slices=50, bits_per_slice=1, num_bits=10):
        super(DivideAndEncode, self).__init__()
        self.num_slices = num_slices
        self.bits_per_slice = bits_per_slice
        self.fc_layers = nn.ModuleList([nn.Linear(num_slices, bits_per_slice) for _ in range(num_bits)])

    def forward(self, x):
        outputs = []
        for fc in self.fc_layers:
            out = F.sigmoid(fc(x))  # Applying sigmoid to restrict output to [0, 1]
            out = self.piecewise_threshold(out)
            outputs.append(out)
        return torch.cat(outputs, dim=1)

    def piecewise_threshold(self, s, epsilon=0.05):
        return torch.where(s < 0.5 - epsilon, torch.zeros_like(s),
                           torch.where(s > 0.5 + epsilon, torch.ones_like(s), s))


## Triplet Ranking Loss

In [5]:
class TripletRankingLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletRankingLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = (anchor - positive).pow(2).sum(1)  # L2 squared
        distance_negative = (anchor - negative).pow(2).sum(1)  # L2 squared
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()


## Training

In [6]:
class IntegratedModel(nn.Module):
    def __init__(self, num_bits):
        super(IntegratedModel, self).__init__()
        self.shared_subnet = SharedSubNet()
        self.divide_and_encode = DivideAndEncode(num_slices=50, bits_per_slice=1, num_bits=num_bits)
        self.fc = nn.Linear(24, 512)  # Adjusted input size
        self.relu = nn.ReLU()
        self.final_fc = nn.Linear(512, num_bits)  # Output bits as the number of output features

    def forward(self, x):
        x = self.shared_subnet(x)
        print("Shape after SharedSubNet:", x.shape)
        x = self.divide_and_encode(x)
        print("Shape after DivideAndEncode:", x.shape)
        x = self.fc(x)
        x = self.relu(x)
        x = self.final_fc(x)
        x = torch.tanh(x)
        x = F.normalize(x, p=2, dim=1)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Model, optimizer, and loss
model = IntegratedModel(input_dim=X_train_tensor.shape[1], num_bits=36).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_func = TripletRankingLoss()

for epoch in range(20):
    model.train()
    total_loss = 0.0
    for anchor, positive, negative, _, _ in train_loader:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        optimizer.zero_grad()
        anchor_output = model(anchor)
        positive_output = model(positive)
        negative_output = model(negative)
        loss = loss_func(anchor_output, positive_output, negative_output)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')

Epoch 1, Loss: 0.29255810560024453
Epoch 2, Loss: 0.2240759517465319
Epoch 3, Loss: 0.20614353708748115
Epoch 4, Loss: 0.19534982903711579
Epoch 5, Loss: 0.18629102719723967
Epoch 6, Loss: 0.17783446526571886
Epoch 7, Loss: 0.1719017482091962
Epoch 8, Loss: 0.1656867018513588
Epoch 9, Loss: 0.16129774056698043
Epoch 10, Loss: 0.15612722212328248
Epoch 11, Loss: 0.15022669756399798
Epoch 12, Loss: 0.14640292532704957
Epoch 13, Loss: 0.14265342447814064
Epoch 14, Loss: 0.13811141284819436
Epoch 15, Loss: 0.13537216856652062
Epoch 16, Loss: 0.13042225370312
Epoch 17, Loss: 0.12829229511405316
Epoch 18, Loss: 0.1258821360629208
Epoch 19, Loss: 0.12210346859771899
Epoch 20, Loss: 0.11818839918342819


## KNN on hashed embeddings

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

def evaluate_model(model, test_loader, device):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for anchor, _, _, label_a, _ in test_loader:  # Correctly unpack all elements
            anchor = anchor.to(device)
            output = model(anchor)
            embeddings.append(output.cpu())
            labels.append(label_a)
    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels

# Ensure model and device are defined and properly initialized
# Example: model = DPSH(input_dim=X_train.shape[1], num_bits=48).to(device)
# and device is defined like device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Extract hash codes
train_codes, train_labels = evaluate_model(model, train_loader, device)
test_codes, test_labels = evaluate_model(model, test_loader, device)

# Classification with KNN
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(train_codes, train_labels)
predictions = knn.predict(test_codes)
y_pred_proba = knn.predict_proba(test_codes)

print(classification_report(test_labels, predictions))

# Binarize the labels for a one-vs-rest computation
y_test_binarized = label_binarize(test_labels, classes=np.unique(train_labels))  # Updated to use `test_labels`

# Calculate the average precision for each class
average_precisions = []
for i in range(y_test_binarized.shape[1]):  # iterate over classes
    average_precisions.append(average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i]))

# Compute the mean of the average precisions
map_score = np.mean(average_precisions)
print(f'Mean Average Precision (MAP): {map_score}')


              precision    recall  f1-score   support

           0       0.94      0.97      0.96      1338
           1       0.93      0.99      0.96       847
           2       0.43      0.63      0.51       339
           3       0.91      0.87      0.89       634
           4       0.90      0.74      0.82      1035
           5       0.63      0.70      0.66       592
           6       0.87      0.77      0.82       741
           7       0.41      0.45      0.43       421
           8       0.84      0.77      0.80      1233

    accuracy                           0.81      7180
   macro avg       0.76      0.77      0.76      7180
weighted avg       0.82      0.81      0.81      7180

Mean Average Precision (MAP): 0.7285328496093608
